In [71]:
!pip install networkx

In [116]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import heapq
import pandas as pd
import json
import numpy as np

In [73]:
import pandas as pd

df = pd.read_excel('./dataset/Data SC ACO.xlsx', sheet_name='Sheet1')

df['TV'] = df['Jarak (KM)'] / df['Kecepatan']

df.head()

# len(df)



,Kode Ruas,Node From,Node To,Kode Jalan,Nama Jalan,Jarak (KM),Vol max (q max),Vol (q),Kecepatan,TV
0,1,257,220,1,Raya Waru,2.101527,1011,962,56.5,0.037195
1,2,220,257,1,Raya Waru,2.101527,1014,962,56.5,0.037195
2,3,192,220,2,Letjend Sutoyo,0.385693,4382,4000,40.0,0.009642
3,4,220,192,2,Letjend Sutoyo,0.385693,4382,3422,40.0,0.009642
4,5,192,149,2,Letjend Sutoyo,0.795825,2124,1658,40.0,0.019896


In [74]:
# import pandas as pd

# df = pd.read_excel('./dataset/Data SC ACO.xlsx', sheet_name='Sheet1')

# df.head()

In [75]:
depot_df = pd.read_csv('./dataset/depot.csv')

depot_df.head()

,Nama,Node,Persediaan,Time Begin,Time End
0,Depo 1,66,450,7:00,15:00
1,Depo 2,277,500,7:00,17:00
2,Depo 3,322,500,7:00,17:00


In [76]:
depot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Nama        3 non-null      object
 1   Node        3 non-null      int64 
 2   Persediaan  3 non-null      int64 
 3   Time Begin  3 non-null      object
 4   Time End    3 non-null      object
dtypes: int64(2), object(3)
memory usage: 248.0+ bytes


In [77]:
customer_df = pd.read_csv('./dataset/customer.csv')

customer_df.head()

,Nama,Node,Permintaan,Time Begin,Time End
0,Konsumen 1,1,25,7:00,8:00
1,Konsumen 2,7,30,7:00,8:00
2,Konsumen 3,12,20,7:00,8:00
3,Konsumen 4,21,35,9:00,9:00
4,Konsumen 5,46,30,8:00,10:00


In [78]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Nama        50 non-null     object
 1   Node        50 non-null     int64 
 2   Permintaan  50 non-null     int64 
 3   Time Begin  50 non-null     object
 4   Time End    50 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.1+ KB


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928 entries, 0 to 927
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Kode Ruas        928 non-null    int64  
 1   Node From        928 non-null    int64  
 2   Node To          928 non-null    int64  
 3   Kode Jalan       928 non-null    int64  
 4   Nama Jalan       928 non-null    object 
 5   Jarak (KM)       928 non-null    float64
 6   Vol max (q max)  928 non-null    int64  
 7   Vol (q)          928 non-null    int64  
 8   Kecepatan        928 non-null    float64
 9   TV               928 non-null    float64
dtypes: float64(3), int64(6), object(1)
memory usage: 72.6+ KB


In [80]:
def time_to_decimal(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours + minutes / 60

depot_df['Time Begin'] = depot_df['Time Begin'].apply(time_to_decimal)
depot_df['Time End'] = depot_df['Time End'].apply(time_to_decimal)
customer_df['Time Begin'] = customer_df['Time Begin'].apply(time_to_decimal)
customer_df['Time End'] = customer_df['Time End'].apply(time_to_decimal)

In [81]:
depot_df

,Nama,Node,Persediaan,Time Begin,Time End
0,Depo 1,66,450,7.0,15.0
1,Depo 2,277,500,7.0,17.0
2,Depo 3,322,500,7.0,17.0


In [82]:
customer_df.head()

,Nama,Node,Permintaan,Time Begin,Time End
0,Konsumen 1,1,25,7.0,8.0
1,Konsumen 2,7,30,7.0,8.0
2,Konsumen 3,12,20,7.0,8.0
3,Konsumen 4,21,35,9.0,9.0
4,Konsumen 5,46,30,8.0,10.0


In [83]:
time_windows = {row['Node']: (row['Time Begin'], row['Time End']) for _, row in depot_df.iterrows()}
time_windows.update({row['Node']: (row['Time Begin'], row['Time End']) for _, row in customer_df.iterrows()})

# Define depots and customers
depots = depot_df['Node'].tolist()
customers = customer_df['Node'].tolist()


print("Time Windows:", time_windows)
print("Depots:", depots)
print("Customers:", customers)

Time Windows: {66: (7.0, 15.0), 277: (7.0, 17.0), 322: (7.0, 17.0), 1: (7.0, 8.0), 7: (7.0, 8.0), 12: (7.0, 8.0), 21: (9.0, 9.0), 46: (8.0, 10.0), 60: (7.0, 10.0), 95: (7.0, 8.0), 135: (7.0, 8.0), 177: (8.0, 11.0), 189: (7.0, 11.0), 210: (8.0, 10.0), 244: (8.0, 11.0), 256: (7.0, 9.0), 289: (8.0, 9.0), 315: (7.0, 8.0), 333: (8.0, 12.0), 376: (8.0, 10.0), 402: (7.0, 10.0), 355: (7.0, 8.0), 400: (7.0, 12.0), 235: (8.0, 9.0), 104: (9.0, 11.0), 144: (11.0, 12.0), 140: (9.0, 11.0), 101: (12.0, 13.0), 254: (14.0, 15.0), 141: (10.0, 12.0), 221: (13.0, 14.0), 174: (9.0, 10.0), 264: (10.0, 11.0), 248: (7.0, 9.0), 298: (7.0, 9.0), 386: (7.0, 9.0), 39: (7.0, 9.0), 81: (7.0, 9.0), 44: (8.0, 10.0), 62: (8.0, 10.0), 28: (8.0, 10.0), 4: (10.0, 10.0), 113: (8.0, 10.0), 106: (9.0, 12.0), 181: (9.0, 12.0), 175: (9.0, 12.0), 205: (9.0, 12.0), 161: (9.0, 12.0), 292: (10.0, 12.0), 242: (10.0, 12.0), 399: (10.0, 12.0), 408: (10.0, 12.0), 296: (10.0, 12.0)}
Depots: [66, 277, 322]
Customers: [1, 7, 12, 21, 46,

In [84]:
G = nx.DiGraph()
# Menambahkan edge dengan atribut jarak dan kecepatan
for idx, row in df.iterrows():
    # print(idx,row['Node From'],row['Node To'],row['Jarak (KM)'],row['TV'])
    G.add_edge(row['Node From'], row['Node To'], 
               distance=row['Jarak (KM)'], speed=row['Kecepatan'], TV = row['TV'])
# print(f"Graph memiliki {G.number_of_nodes()} nodus dan {G.number_of_edges()} ruas jalan.")
print(G)

DiGraph with 400 nodes and 927 edges


In [85]:
# def dijkstra(graph, start_vertex,wtv = 0):
#     distances = {vertex: float('infinity') for vertex in graph}
#     distances[start_vertex] = 0
#     priority_queue = [(0, start_vertex)]

#     while priority_queue:
#         current_distance, current_vertex = heapq.heappop(priority_queue)

#         if current_distance > distances[current_vertex]:
#             continue

#         for neighbor, weight in graph[current_vertex].items():
#             # print(neighbor,weight['weight'],weight['TV'])
#             distance = current_distance + (weight['weight'] if wtv == 0 else weight['TV'])

#             if distance < distances[neighbor]:
#                 distances[neighbor] = distance
#                 heapq.heappush(priority_queue, (distance, neighbor))

#     return distances
# def distance(i, j, graph,wtv = 0):
#     distances = dijkstra(graph, i,wtv=wtv)
#     return distances[j]
# distance(1, 7, G)

In [86]:
distance = {}

for idx, row in customer_df.iterrows():
    node = int(row['Node'])
    path_lengths = nx.single_source_dijkstra_path_length(G, node, weight='distance')
    distance[node] = {dest: dist for dest, dist in path_lengths.items()}
for idx, row in depot_df.iterrows():
    node = int(row['Node'])
    path_lengths = nx.single_source_dijkstra_path_length(G, node, weight='distance')
    distance[node] = {dest: dist for dest, dist in path_lengths.items()}
    
with open('./dataset/distances.json', 'w') as f:
    json.dump(distance, f, indent=4)


with open('./dataset/distances.json', 'r') as f:
    distance = json.load(f)


In [87]:
distance = {}

for idx, row in customer_df.iterrows():
    node = int(row['Node'])
    path_lengths = nx.single_source_dijkstra_path_length(G, node, weight='TV')
    distance[node] = {dest: dist for dest, dist in path_lengths.items()}
for idx, row in depot_df.iterrows():
    node = int(row['Node'])
    path_lengths = nx.single_source_dijkstra_path_length(G, node, weight='TV')
    distance[node] = {dest: dist for dest, dist in path_lengths.items()}
    
with open('./dataset/time.json', 'w') as f:
    json.dump(distance, f, indent=4)


with open('./dataset/time.json', 'r') as f:
    time = json.load(f)

In [88]:
# graph = nx.DiGraph()
# for index, row in df.iterrows():
#     # print(row['Node From'],row['Node To'],row['Jarak (KM)'],row['TV'])
#     graph.add_edge(row['Node From'], row['Node To'], 
#                    weight=row['Jarak (KM)'], 
#                    TV=row['TV'])

In [89]:
# path_1 = nx.dijkstra_path(G, 66, 60, weight='weight')
# path_length_1 = nx.dijkstra_path_length(G, 66, 60, weight='distance')
# path_2 = nx.dijkstra_path(G, 277, 60, weight='weight')
# path_length_2 = nx.dijkstra_path_length(G, 277, 60, weight='distance')
# path_3 = nx.dijkstra_path(G, 322, 60, weight='weight')
# path_length_3 = nx.dijkstra_path_length(G, 322, 60, weight='distance')


# print(distance(66, 60, graph))
# print(distance(277, 60, graph))
# print(distance(322, 60, graph))
# print("Shortest path:", path_1)
# print("Path length:", path_length_1)

# print("Shortest path:", path_2)
# print("Path length:", path_length_2)


# print("Shortest path:", path_3)
# print("Path length:", path_length_3)

In [90]:
def find_two_nearest_depots(customer, depots):
    dis = []
    for i in depots:
        dis.append((distance[str(customer)][str(i)], i))
    dis.sort(key=lambda x: x[0])
    # print(dis)
    return [dis[0][1], dis[1][1]]

In [91]:
find_two_nearest_depots(60,depots)

[66, 277]

In [92]:
find_two_nearest_depots(254,depots)

[322, 66]

In [93]:
def find_init_customers(depots, customers):
    C = {}
    for i in depots:
        mn = (float('inf'), None)
        for j in customers:
            if distance[str(i)][str(j)] < mn[0]:
                mn = (distance[str(i)][str(j)], j)
        C[i] = [mn[1]]
    return C

In [94]:
def calculate_dtw(i, j, time_windows):
    li , ei = time_windows[i]
    lj, ej = time_windows[j]
    if li < ej:
        return ej - li
    elif lj < ei:
        return ei - lj
    else:
        return 0

In [95]:
def calculate_affinity(i, depots, customer_to_depot, time_windows):
    affinities = {}
    for d in depots:
        associated_customers = customer_to_depot[d]
        affinity_sum = 0
        for j in associated_customers:
            dtw = calculate_dtw(i, j, time_windows)
            tij = time[str(i)][str(j)]
            affinity_sum += np.exp(-(dtw + tij))
        affinities[d] = affinity_sum / len(associated_customers) if associated_customers else 0
    return affinities

In [96]:
def calculate_closeness(i, j, affinity):
    return distance[str(i)][str(j)] / affinity

In [97]:
def determine_urgency(customers, depots,customer_to_depot):
    urgencies = {}
    for c in customers:
        nearest_two_depots = find_two_nearest_depots(c, depots)
        # print(nearest_two_depots)
        # print(distance[str(nearest_two_depots[0])][str(c)], distance[str(nearest_two_depots[1])][str(c)])
        if len(nearest_two_depots) >= 2:
            affiniti = calculate_affinity(c,depots=depots,customer_to_depot=customer_to_depot,time_windows=time_windows)    
            urgencies[c] = calculate_closeness(nearest_two_depots[0],c, affiniti[nearest_two_depots[0]]) - calculate_closeness(nearest_two_depots[1],c, affiniti[nearest_two_depots[1]])
        # break
    sorted_dict = dict(sorted(urgencies.items(), key=lambda item: item[1],reverse=True))

    print(sorted_dict)
    max_urgent = max(urgencies, key=urgencies.get)
    return (max_urgent,urgencies[max_urgent])

In [98]:
C = find_init_customers(depots,customers=customers)
C

{66: [62], 277: [296], 322: [376]}

In [99]:
urgency = determine_urgency(customers=customers, depots=depots,customer_to_depot=C)
urgency

{95: np.float64(433.8505546342192), 135: np.float64(428.26997036853356), 221: np.float64(124.10777767108857), 244: np.float64(65.09917344693032), 106: np.float64(64.14290373551819), 175: np.float64(59.398751437070885), 181: np.float64(37.85544052246121), 161: np.float64(26.935155651521), 205: np.float64(17.20686940189992), 210: np.float64(3.2274494720371933), 140: np.float64(-1.0328360495077202), 400: np.float64(-4.110901459511169), 174: np.float64(-4.40023103454501), 248: np.float64(-8.284978107935075), 289: np.float64(-11.654499123211497), 104: np.float64(-12.339504138347685), 177: np.float64(-14.273676455323855), 264: np.float64(-17.288722162028947), 386: np.float64(-25.410286417964315), 333: np.float64(-29.242688145109422), 4: np.float64(-30.237460564883285), 113: np.float64(-30.513354602101604), 189: np.float64(-32.24670783362082), 298: np.float64(-40.352303051211855), 402: np.float64(-49.02365859288387), 376: np.float64(-50.13857178959036), 315: np.float64(-50.16459608176222), 23

(95, np.float64(433.8505546342192))

In [100]:
temp_customer=customers.copy()

In [101]:

persediaan = {row['Node']: row['Persediaan'] for _, row in depot_df.iterrows()}
for key, value in C.items():
    temp_customer.remove(value[0])
while temp_customer:
    # temp_depots =depots.copy()
    indx , max_urgensi_values = determine_urgency(customers=temp_customer, depots=depots,customer_to_depot=C)
    print(indx,max_urgensi_values)
    first, second = find_two_nearest_depots(indx,depots)
    print(first,second)
    permintaan =  int(customer_df[customer_df['Node'] == indx]['Permintaan'].iloc[0])

    if persediaan[first] >= permintaan:
        C[first].append(indx)
        persediaan[first] -= permintaan
    elif persediaan[second] >= permintaan:
        C[second].append(indx)
        persediaan[second] -= permintaan
    else:
        for i in depots:
            if i != first and i != second:
                C[i].append(indx)
        
    print("Permintaan : ", permintaan)
    for i in persediaan:
        print(persediaan[i],type(persediaan[i]))
    print(temp_customer)
    temp_customer.remove(indx)
    print(temp_customer)
    print("C", C)
    # break

{95: np.float64(433.8505546342192), 135: np.float64(428.26997036853356), 221: np.float64(124.10777767108857), 244: np.float64(65.09917344693032), 106: np.float64(64.14290373551819), 175: np.float64(59.398751437070885), 181: np.float64(37.85544052246121), 161: np.float64(26.935155651521), 205: np.float64(17.20686940189992), 210: np.float64(3.2274494720371933), 140: np.float64(-1.0328360495077202), 400: np.float64(-4.110901459511169), 174: np.float64(-4.40023103454501), 248: np.float64(-8.284978107935075), 289: np.float64(-11.654499123211497), 104: np.float64(-12.339504138347685), 177: np.float64(-14.273676455323855), 264: np.float64(-17.288722162028947), 386: np.float64(-25.410286417964315), 333: np.float64(-29.242688145109422), 4: np.float64(-30.237460564883285), 113: np.float64(-30.513354602101604), 189: np.float64(-32.24670783362082), 298: np.float64(-40.352303051211855), 402: np.float64(-49.02365859288387), 315: np.float64(-50.16459608176222), 235: np.float64(-52.537586684586884), 3

In [102]:
print(C)

{66: [62, 101, 1, 141, 113, 60, 4, 189, 174, 7, 256, 333, 235, 12, 21, 81, 39], 277: [296, 95, 106, 244, 175, 181, 161, 135, 205, 242, 408, 399, 292, 44, 28, 46, 254], 322: [376, 221, 248, 144, 400, 386, 140, 104, 402, 298, 264, 315, 210, 289, 355, 177]}


In [103]:
depos = {
    "1": [189, 174, 113, 46, 62, 60, 141, 81, 44, 39, 28, 140,
               101, 4, 21, 254, 12, 7, 235, 144, 1, 104],
    "2": [408, 95, 399, 296, 135, 292, 242, 205, 244, 161, 181, 175,
               106],
    "3": [210, 386, 248, 289, 298, 264, 221, 355, 256, 315, 402, 400,
               333, 177, 376]
}
print(sorted(depos["1"]))
print(sorted(depos["2"]))
print(sorted(depos["3"]))

print(len(depos["1"])+len(depos["2"])+len(depos["3"]))


for i in depos:
    count = 0 
    for j in depos[i]:
        count += int(customer_df[customer_df['Node'] == j]['Permintaan'].iloc[0])
    print(i,count)

[1, 4, 7, 12, 21, 28, 39, 44, 46, 60, 62, 81, 101, 104, 113, 140, 141, 144, 174, 189, 235, 254]
[95, 106, 135, 161, 175, 181, 205, 242, 244, 292, 296, 399, 408]
[177, 210, 221, 248, 256, 264, 289, 298, 315, 333, 355, 376, 386, 400, 402]
50
1 605
2 300
3 485


In [104]:
print(sorted(C[66]))
print(sorted(C[277]))
print(sorted(C[322]))
count =0 
for i in C:
    for j in C[i]:
        if (i == 66 and j in depos["1"]) or (i == 277 and j in depos["2"]) or (i == 322 and j in depos["3"]):
            count+=1
print(count)
print(len(C[66])+len(C[277])+len(C[322]))   

[1, 4, 7, 12, 21, 39, 60, 62, 81, 101, 113, 141, 174, 189, 235, 256, 333]
[28, 44, 46, 95, 106, 135, 161, 175, 181, 205, 242, 244, 254, 292, 296, 399, 408]
[104, 140, 144, 177, 210, 221, 248, 264, 289, 298, 315, 355, 376, 386, 400, 402]
41
50


In [120]:
D  = {66: [62, 101, 1, 141, 113, 60, 4, 189, 174, 7, 256, 333, 235, 12, 21, 81, 39], 277: [296, 95, 106, 244, 175, 181, 161, 135, 205, 242, 408, 399, 292, 44, 28, 46, 254], 322: [376, 221, 248, 144, 400, 386, 140, 104, 402, 298, 264, 315, 210, 289, 355, 177]}
count = 0
for i in C:
    for j in C[i]:
        if (i == 66 and j in depos["1"]) or (i == 277 and j in depos["2"]) or (i == 322 and j in depos["3"]):
            count+=1
print(count)

41


In [105]:
import pandas as pd
import networkx as nx
import numpy as np
import heapq

def dijkstra(graph, start_vertex,wtv = 0):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start_vertex] = 0
    priority_queue = [(0, start_vertex)]

    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            # print(neighbor,weight['weight'],weight['TV'])
            distance = current_distance + (weight['weight'] if wtv == 0 else weight['TV'])

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

def distance(i, j, graph,wtv = 0):
    distances = dijkstra(graph, i,wtv=wtv)
    return distances[j]


def find_nearest_customers(depots, customers, graph):
    associations = {}
    for d in depots:
        min_distance = float('infinity')
        nearest_customer = None
        for c in customers:
            dist = distance(d, c, graph)
            if dist < min_distance:
                min_distance = dist
                nearest_customer = c
        if nearest_customer is not None:
            associations[nearest_customer] = d
    # print(associations)
    associations = {value: [key] for key, value in associations.items()}
    return associations

def find_two_nearest_depots(customer, depots, graph):
    distances = [(depot, distance(depot, customer, graph)) for depot in depots]
    # print(distances)
    sorted_depots = sorted(distances, key=lambda x: x[1])
    # print([depot for depot in sorted_depots[:]])
    return [depot[0] for depot in sorted_depots[:2]]

def calculate_dtw(i, j, time_windows):
    li , ei = time_windows[i]
    if i ==135:
        print(time_windows[i],time_windows[j])
    lj, ej = time_windows[j]
    if li < ej:
        if i ==135:
            print(li,ej)
            print("masuk 1")
        return ej - li
    elif lj < ei:
        if i == 135:
            print("masuk 2")
        return ei - lj
    else:
        if i == 135:
            print("lalla")
        return 0
    
def calculate_affinity(i, depots, customer_to_depot, graph, time_windows):
    affinities = {}
    for d in depots:
        # print(d)
        associated_customers = customer_to_depot[d]
        # print(associated_customers)
        affinity_sum = 0
        for j in associated_customers:
            dtw = calculate_dtw(i, j, time_windows)
            tij = distance(i, j, graph,wtv=1)
            if i == 135:
                print("affinity :" ,d,i,j,dtw, tij)
            affinity_sum += np.exp(-(dtw + tij))
            if i == 135:
                print("affinity :" ,affinity_sum)
        affinities[d] = affinity_sum / len(associated_customers) if associated_customers else 0
    return affinities

def calculate_closeness(i, j, graph, affinity):
    return distance(i, j, graph) / affinity

def determine_urgency(customers, depots, closeness, graph):
    urgencies = {}
    for c in customers:
        nearest_two_depots = find_two_nearest_depots(c, depots, graph)
        if len(nearest_two_depots) >= 2:
            closeness_diff = closeness[c][nearest_two_depots[0]] - closeness[c][nearest_two_depots[1]]
            urgencies[c] = closeness_diff
    return urgencies


In [106]:
df

,Kode Ruas,Node From,Node To,Kode Jalan,Nama Jalan,Jarak (KM),Vol max (q max),Vol (q),Kecepatan,TV
0,1,257,220,1,Raya Waru,2.101527,1011,962,56.5,0.037195
1,2,220,257,1,Raya Waru,2.101527,1014,962,56.5,0.037195
2,3,192,220,2,Letjend Sutoyo,0.385693,4382,4000,40.0,0.009642
3,4,220,192,2,Letjend Sutoyo,0.385693,4382,3422,40.0,0.009642
4,5,192,149,2,Letjend Sutoyo,0.795825,2124,1658,40.0,0.019896
...,...,...,...,...,...,...,...,...,...,...
923,924,333,339,83,Darmo Kali,0.327487,5161,4030,51.5,0.006359
924,925,333,319,83,Darmo Kali,0.574090,2944,2299,51.5,0.011147
925,926,319,333,83,Darmo Kali,0.574090,2944,2299,51.5,0.011147
926,927,319,305,83,Darmo Kali,0.229289,7372,5756,51.5,0.004452


In [107]:
# df = pd.read_excel('./dataset/Data SC ACO.xlsx', sheet_name='Sheet1')
graph = nx.DiGraph()
for index, row in df.iterrows():
    print(row['Node From'],row['Node To'],row['Jarak (KM)'],row['TV'])
    graph.add_edge(row['Node From'], row['Node To'], 
                   weight=row['Jarak (KM)'], 
                   TV=row['TV'])

257 220 2.1015272 0.03719517168141593
220 257 2.1015272 0.03719517168141593
192 220 0.3856929 0.0096423225
220 192 0.3856929 0.0096423225
192 149 0.7958252 0.01989563
149 192 0.7958252 0.01989563
149 124 0.4301281 0.0107532025
124 149 0.4301281 0.0107532025
220 222 1.1688458 0.02068753628318584
222 220 1.1688458 0.02068753628318584
222 229 0.4786694 0.00828147750865052
229 235 0.5578447 0.009651292387543253
235 377 0.3682297 0.006370756055363322
377 236 0.1056195 0.0018273269896193773
236 222 1.2992554 0.022478467128027684
377 378 0.6552817 0.011337053633217993
378 377 0.6552817 0.011337053633217993
378 247 0.244266 0.0042260553633218
247 251 0.0611755 0.0010583996539792388
251 378 0.3053636 0.0052831072664359865
251 245 0.3997919 0.006916814878892733
245 263 0.2864912 0.004956595155709343
263 258 0.3161118 0.0054690622837370246
258 251 0.3379633 0.005847115916955018
263 379 0.8223246 0.014227069204152249
379 263 0.8223246 0.014227069204152249
379 285 1.1028258 0.01908003114186851
286 

In [108]:
print(graph)

DiGraph with 400 nodes and 927 edges


In [109]:
time_windows = {row['Node']: (row['Time Begin'], row['Time End']) for _, row in depot_df.iterrows()}
time_windows.update({row['Node']: (row['Time Begin'], row['Time End']) for _, row in customer_df.iterrows()})

# Define depots and customers
depots = depot_df['Node'].tolist()
customers = customer_df['Node'].tolist()


print("Time Windows:", time_windows)
print("Depots:", depots)
print("Customers:", customers)

Time Windows: {66: (7.0, 15.0), 277: (7.0, 17.0), 322: (7.0, 17.0), 1: (7.0, 8.0), 7: (7.0, 8.0), 12: (7.0, 8.0), 21: (9.0, 9.0), 46: (8.0, 10.0), 60: (7.0, 10.0), 95: (7.0, 8.0), 135: (7.0, 8.0), 177: (8.0, 11.0), 189: (7.0, 11.0), 210: (8.0, 10.0), 244: (8.0, 11.0), 256: (7.0, 9.0), 289: (8.0, 9.0), 315: (7.0, 8.0), 333: (8.0, 12.0), 376: (8.0, 10.0), 402: (7.0, 10.0), 355: (7.0, 8.0), 400: (7.0, 12.0), 235: (8.0, 9.0), 104: (9.0, 11.0), 144: (11.0, 12.0), 140: (9.0, 11.0), 101: (12.0, 13.0), 254: (14.0, 15.0), 141: (10.0, 12.0), 221: (13.0, 14.0), 174: (9.0, 10.0), 264: (10.0, 11.0), 248: (7.0, 9.0), 298: (7.0, 9.0), 386: (7.0, 9.0), 39: (7.0, 9.0), 81: (7.0, 9.0), 44: (8.0, 10.0), 62: (8.0, 10.0), 28: (8.0, 10.0), 4: (10.0, 10.0), 113: (8.0, 10.0), 106: (9.0, 12.0), 181: (9.0, 12.0), 175: (9.0, 12.0), 205: (9.0, 12.0), 161: (9.0, 12.0), 292: (10.0, 12.0), 242: (10.0, 12.0), 399: (10.0, 12.0), 408: (10.0, 12.0), 296: (10.0, 12.0)}
Depots: [66, 277, 322]
Customers: [1, 7, 12, 21, 46,

In [110]:
nearest_customers = find_nearest_customers(depots, customers, graph)
print("Nearest Customers:", nearest_customers)
for i in nearest_customers:
    print(distance(i,nearest_customers[i][0],graph=graph))
affinities = {c: calculate_affinity(c, depots, nearest_customers, graph, time_windows) for c in customers}
# print("Affinities:", affinities)
closeness = {c: {d: calculate_closeness(c, d, graph, affinities[c][d]) for d in depots} for c in customers}
# print("Closeness:", closeness)
urgencies = determine_urgency(customers, depots, closeness,graph)
# print("Urgencies:", urgencies)
persediaan = {row['Node']: row['Persediaan'] for _, row in depot_df.iterrows()}
# for key, value in nearest_customers.items():
    
temp_customer=customers   
# sorted_dict = {k: v for k, v in sorted(urgencies.items(), key=lambda item: item[1], reverse=True)}

# for c in sorted_dict:
#     print(c,sorted_dict[c])
#     # customer_urgent = temp_customer.pop(c)

for key, value in nearest_customers.items():
    temp_customer.remove(value[0])

while temp_customer:
    highest_urgency_key = max(urgencies, key=urgencies.get)
    sorted_dict = {k: v for k, v in sorted(urgencies.items(), key=lambda item: item[1], reverse=True)}
    print(sorted_dict)
    print(highest_urgency_key)
    first , second = find_two_nearest_depots(highest_urgency_key,depots,graph)
    print(first,second)
    print(distance(highest_urgency_key,first,graph), distance(highest_urgency_key,second,graph))
    del urgencies[highest_urgency_key]
    permintaan =  int(customer_df[customer_df['Node'] == highest_urgency_key]['Permintaan'].iloc[0])
    # filtered_df = customer_df[customer_df['Node'] == highest_urgency_key]
    # print(filtered_df)
    for i in persediaan:
        print(persediaan[i],type(persediaan[i]))
    # print(persediaan[first],type(persediaan[first]))
    # print(persediaan[second],type(persediaan[second]))
    print(permintaan,type(permintaan))
    if persediaan[first] > permintaan:

        persediaan[first] -= permintaan
        nearest_customers[first].append(highest_urgency_key)
    else:       
        persediaan[second] -= permintaan
        nearest_customers[second].append(highest_urgency_key)
    if highest_urgency_key in temp_customer:
        print("IYOOOOOOOOOOOOO")
    print(nearest_customers)
    temp_customer.remove(highest_urgency_key)
    affinities = {c: calculate_affinity(c, depots, nearest_customers, graph, time_windows) for c in temp_customer}
    # print("Affinities:", affinities)
    closeness = {c: {d: calculate_closeness(c, d, graph, affinities[c][d]) for d in depots} for c in temp_customer}
    # print("Closeness:", closeness)
    urgencies = determine_urgency(temp_customer, depots, closeness,graph)
    print(persediaan[first],type(persediaan[first]))
    print(persediaan[second],type(persediaan[second]))
    print(first,second)
    print(temp_customer)
    print()
    # break
    
    

    
print("Nearest Customers:", nearest_customers)
# print("Affinities:", affinities)
# print("Closeness:", closeness)
# print("Urgencies:", urgencies)

Nearest Customers: {66: [62], 277: [296], 322: [376]}
0.2739541
0.9892962999999999
1.2873336
(7.0, 8.0) (8.0, 10.0)
7.0 10.0
masuk 1
affinity : 66 135 62 3.0 0.10992340522727273
affinity : 0.04460437167114711
(7.0, 8.0) (10.0, 12.0)
7.0 12.0
masuk 1
affinity : 277 135 296 5.0 0.06959260587246702
affinity : 0.006284980083290539
(7.0, 8.0) (8.0, 10.0)
7.0 10.0
masuk 1
affinity : 322 135 376 3.0 0.1496194363930793
affinity : 0.04286843793050663
{60: np.float64(793.3659279493885), 135: np.float64(632.3429002748587), 95: np.float64(575.4057721986917), 221: np.float64(344.90955607151955), 141: np.float64(135.0943227944386), 244: np.float64(92.24186240379728), 175: np.float64(86.7839558026357), 106: np.float64(83.39640010069338), 181: np.float64(64.74844261746044), 161: np.float64(52.21313264832517), 205: np.float64(39.171618431797526), 210: np.float64(5.402215955017667), 140: np.float64(-3.38473759592161), 104: np.float64(-9.116122139936266), 289: np.float64(-17.93152958115062), 177: np.floa

In [111]:
print()

In [112]:
depos = {
    "1": [189, 174, 113, 46, 62, 60, 141, 81, 44, 39, 28, 140,
               101, 4, 21, 254, 12, 7, 235, 144, 1, 104],
    "2": [408, 95, 399, 296, 135, 292, 242, 205, 244, 161, 181, 175,
               106],
    "3": [210, 386, 248, 289, 298, 264, 221, 355, 256, 315, 402, 400,
               333, 177, 376]
}

In [113]:
print(sorted(depos["1"]))
print(sorted(depos["2"]))
print(sorted(depos["3"]))

[1, 4, 7, 12, 21, 28, 39, 44, 46, 60, 62, 81, 101, 104, 113, 140, 141, 144, 174, 189, 235, 254]
[95, 106, 135, 161, 175, 181, 205, 242, 244, 292, 296, 399, 408]
[177, 210, 221, 248, 256, 264, 289, 298, 315, 333, 355, 376, 386, 400, 402]


In [114]:
print(sorted(nearest_customers[66]))
print(sorted(nearest_customers[277]))
print(sorted(nearest_customers[322]))
count = 0
for i in nearest_customers:
    for j in nearest_customers[i]:
        if (i == 66 and j in depos["1"]) or (i == 277 and j in depos["2"]) or (i == 322 and j in depos["3"]):
            count+=1
print(count)
print(len(nearest_customers[66]+nearest_customers[277]+nearest_customers[322]))

[1, 4, 7, 12, 21, 39, 46, 62, 81, 141, 174, 189, 210, 221, 235, 333]
[28, 44, 60, 95, 106, 113, 135, 161, 175, 181, 205, 242, 244, 292, 296, 399, 408]
[101, 104, 140, 144, 177, 248, 254, 256, 264, 289, 298, 315, 355, 376, 386, 400, 402]
38
50


In [115]:
for i in depots:
    mn = float('inf')
    for j in customers:
        mn = min(mn, distance(i,j,graph=graph))
        print("distance ",i,j,distance(i,j,graph=graph))

    print("MINIMUM DISTANCE: " , mn)

MINIMUM DISTANCE:  inf
MINIMUM DISTANCE:  inf
MINIMUM DISTANCE:  inf
